In [1]:
import tweepy
import nltk
import pandas as pd
from datetime import datetime, timedelta, date
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from flask import Flask, request, jsonify


/Users/gbJaDaCr/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Input the important user credentials, ideally these would be encrypted

In [2]:

TWITTER_X_CONSUMER_KEY = 'AaHDae8jAkG3H9Ku2eO6hTf7G'
TWITTER_X_CONSUMER_SECRET = 'RYiH18DXSLAlV8FDMDo7Jxc6RMGzHRNoY8iTXnT1EBU3I24eTu'
TWITTER_X_ACCESS_TOKEN = '1701967006843449345-lSjaTVnM288fCWRoDWfIho6zHgZOT5'
TWITTER_X_ACCESS_TOKEN_SECRET = 'F6vLYUwtob9b5zVyRKeIK1fAom2vnxzUVdPnoJVyxFIbz'

TWITTER_X_BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAD69pwEAAAAACugyBA1idqZMcGuK5jsznQpG9Qk%3DYmoIAEmRLLClkvDTjdcwx4ir6dJ4ZarMWZ5iOhLFhORaxJ0aEP'

FLASK_APP_NAME = ''

In [3]:
TEXTBLOB_MODEL_VERSION = 1.0
VADER_MODEL_VERSION = 1.0

FLASK_APP_NAME = ''

NUM_TWEETS_TO_ANALYSE_DEFAULT = 10

In [4]:
# Set Up Twitter/X Authorisation Client for all further calls.      
client = tweepy.Client(TWITTER_X_BEARER_TOKEN, return_type=dict)

# Get useful components from the Natural Language Toolkit for Pre-Prep of Tweets
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Due to limited time, I have chosen to keep to English language only
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# I thought it prudent to show a multi-model solution, initialises the vader analysis
vader_analyzer = SentimentIntensityAnalyzer()

# Create a Flask App
app = Flask(__name__)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gbJaDaCr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/gbJaDaCr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gbJaDaCr/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
def make_df_of_raw_tweet_text(hashtag, num_tweets_to_analyse):
    tweet_dict = client.search_recent_tweets(query=hashtag, max_results=num_tweets_to_analyse)
    tweet_list = []
    for raw_tweet_data in tweet_dict['data']:
        raw_tweet_text = raw_tweet_data['text']
        tweet_list.append(raw_tweet_text)
    df_tweets_raw = pd.DataFrame(data=tweet_list, columns=['tweet_text_raw'])
    df_tweets_raw['request_timestamp'] = datetime.utcnow().isoformat()
    return df_tweets_raw

In [6]:
def preprocess_text(raw_tweet):
    lower_raw_tweet = raw_tweet.lower()
    tokens = word_tokenize(lower_raw_tweet)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalnum()]
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

In [7]:
def perform_textblob_sentiment_analysis(processed_tweet):
    analysis = TextBlob(processed_tweet)
    # Classify sentiment (positive, negative, or neutral)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

In [8]:
def perform_vader_sentiment_analysis(processed_tweet):
    sentiment_scores = vader_analyzer.polarity_scores(processed_tweet)
    return sentiment_scores

In [9]:
def make_df_with_analyses(df_tweets_raw):
    # Pre-Process Tweet texts
    for index, tweet in df_tweets_raw.iterrows():
        processed_text = preprocess_text(tweet['tweet_text_raw'])
        textblob_sentiment_score = perform_textblob_sentiment_analysis(processed_text)
        vader_sentiment_score = perform_vader_sentiment_analysis(processed_text)
        df_tweets_raw.loc[index, 'cleaned_tweet_text'] = processed_text
        df_tweets_raw.loc[index, 'textblob_score'] = textblob_sentiment_score
        df_tweets_raw.loc[index, 'textblob_version'] = TEXTBLOB_MODEL_VERSION
        df_tweets_raw.loc[index, 'neg_vader_score'] = vader_sentiment_score['neg']
        df_tweets_raw.loc[index, 'neu_vader_score'] = vader_sentiment_score['neu']
        df_tweets_raw.loc[index, 'pos_vader_score'] = vader_sentiment_score['pos']
        df_tweets_raw.loc[index, 'comp_vader_score'] = vader_sentiment_score['compound']
        df_tweets_raw.loc[index, 'vader_version'] = VADER_MODEL_VERSION
    # Remove Duplicates
    df_tweets_cleaned = df_tweets_raw.drop_duplicates(ignore_index=True, inplace=False)
    return df_tweets_cleaned

In [10]:
@app.route('/vader_sentiment', methods=['GET'])
def get_sentiment():
    hashtag = request.args.get('hashtag', default=None)
    num_tweets_to_analyse = request.args.get('num_tweets', default=NUM_TWEETS_TO_ANALYSE_DEFAULT)
    if hashtag is None:
        return jsonify({"error": "Please provide a hashtag parameter"}), 400

    # Collect tweets and calculate sentiment
    df_tweets_raw = make_df_of_raw_tweet_text(hashtag, num_tweets_to_analyse)
    df_tweets_sentiments = make_df_with_analyses(df_tweets_raw)
    overall_sentiment_score = round(df_tweets_sentiments['comp_vader_score'].mean(), 3)

    return jsonify({"sentiment_score": overall_sentiment_score})

In [11]:
@app.route('/get_data', methods=['GET'])
def get_sentiment_df():
    hashtag = request.args.get('hashtag', default=None)
    num_tweets_to_analyse = request.args.get('num_tweets', default=NUM_TWEETS_TO_ANALYSE_DEFAULT)
    if hashtag is None:
        return jsonify({"error": "Please provide a hashtag parameter"}), 400

    # Collect tweets and calculate sentiment
    df_tweets_raw = make_df_of_raw_tweet_text(hashtag, num_tweets_to_analyse)
    df_tweets_sentiments = make_df_with_analyses(df_tweets_raw)

    return jsonify(df_tweets_sentiments.to_dict())

In [12]:
app.run(port=8000, debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
/var/folders/tc/_w0nx0p9249bjlgz3sx10kbw0000gp/T/ipykernel_50664/3651679784.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'rt cacustoms clonearmycustoms captain howzer http clonearmycustoms cac lego legostarwars starwars custom' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_tweets_raw.loc[index, 'cleaned_tweet_text'] = processed_text
/var/folders/tc/_w0nx0p9249bjlgz3sx10kbw0000gp/T/ipykernel_50664/3651679784.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'neutral' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_tweets_raw.loc[index, 'textblob_score'] = textblob_sentiment_score
127.0.0.1 - - [18/Sep/2023 00:18:17] "GET /get_data?hashtag=LEGOstarwars&num_tweets=100 HTTP/1.1" 200 -
